In [ ]:

import warnings
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt


warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Read in the data
df = pd.read_csv('xyz-billing.csv')
df['month'] =  pd.to_datetime(df['date']).dt.to_period('M')



Question: Produce either a visualization or table to show the total monthly MRR for January
2021 to June 2022 as at each month end.

Visualisation Analysis:

- Data sample is very small to determine meaningful trends such as seasonality.
- Information on the graph may be enhanced by adding another plot of total revenue(including cal purchases).

In [ ]:
# Firstly, create the main dataframe of the workbook called main_df.Which is used for all the quesitons.
# Mainly,we expand the annual subscriptions into 12 months and divide the amount by 12.
# We then filter out data after July 2022 as the dataset only exists up until June 2022.

# We now make the first dataframe to answer question 1 called revenue_df off the main_df.
# We filter out the calendar purchases(as we only care about MRR)
# We then group by the month and sum the amount to get the revenue per month.
# We then plot the graph.

list_of_dicts = []


for index, row in df.iterrows():
    if row['itemDescription'] == 'annual subscription':
         for i in range(0, 12):
                list_of_dicts.append({'customerid': row['customerID'], 'sub_month': row['month'] + (i+1), 'subscription': row['itemDescription'], 'sub_amount': row['amount']/12})
    else:
         list_of_dicts.append({'customerid': row['customerID'], 'sub_month': row['month'], 'subscription': row['itemDescription'], 'sub_amount': row['amount']})


main_df = pd.DataFrame(list_of_dicts) 
main_df = main_df[main_df['sub_month'] < '2022-07']

revenue_df = main_df
revenue_df = revenue_df[(revenue_df['subscription'] != 'calendar purchase')]
revenue_df = revenue_df.groupby(['sub_month'])['sub_amount'].sum().reset_index()


plt.plot(revenue_df.index, revenue_df['sub_amount'])
plt.xlabel('Month')
plt.ylabel('MRR $')
plt.title('MRR')
plt.xticks(range(0, len(revenue_df.index)), revenue_df['sub_month'], rotation=90)
plt.grid()
plt.show()





Quesiton: Produce a visualization or table to show the monthly growth rates of MRR and
calendar purchases (as separate categories). The monthly growth rate is the defined
as the percentage change from the previous month.

Visualisation Analysis:

-  Monthly growth rates seems more volatile for Calendar
-  Calendar growth seems to lag the MRR


In [ ]:

# We make another dataframe to answer question 2 called revenue_df_split off the main_df.
# We keep regroup subscription into two categories: MRR and CAL.
# We then group by the month and category and sum the amount to get the revenue per month.
# We then pivot the table to get the desired format.
# We then calculate the growth rate by taking the percentage change of the revenue per month.
# We then plot the graph.

revenue_df_split = main_df
revenue_df_split['category'] = revenue_df_split['subscription'].apply(lambda x: 'CAL' if x == 'calendar purchase' else 'MRR')
revenue_df_split = revenue_df_split.groupby(['sub_month', 'category'])['sub_amount'].sum().reset_index()


revenue_df_split = revenue_df_split.pivot(index='sub_month', columns='category', values='sub_amount')
revenue_df_split['cal_growth_rate'] = revenue_df_split['CAL'].pct_change()*100
revenue_df_split['mrr_growth_rate'] = revenue_df_split['MRR'].pct_change()*100
revenue_df_split.fillna(0, inplace=True)
revenue_df_split.reset_index(inplace=True)


plt.plot(revenue_df_split.index, revenue_df_split['cal_growth_rate'], label='CAL')
plt.plot(revenue_df_split.index, revenue_df_split['mrr_growth_rate'], label='MRR')
plt.xlabel('Month')
plt.ylabel('Growth Rate %')
plt.title('Monthly Revenue Growth Rates')
plt.xticks(range(0, len(revenue_df_split.index)), revenue_df_split['sub_month'], rotation=90)
plt.grid()
plt.legend()
plt.show()



Question: As a SaaS it’s important to identify when customers are churning, produce a table or
visualization to identify how many customers have churned each month between
December 2021 and March 2022 (both months inclusive) where churn is defined as a
customer having a non zero-MRR amount attributed in the previous month but no
MRR in the current month.

Visualisation Analysis:

- Difficult to pull out insights due to limited date range.
- Ideally a weighting should be applied to get the relative impact of customers churning.
- Possible data integrity(exceptions) such as customer 13 which only has one entry for a monthly subscription. 

In [ ]:

# we make another dataframe to answer question 3 called customer_df off the main_df.
# We filter out the calendar purchases(as we only care about MRR).
# We then get the unique customerid and sub_month out of the dataset
# We then create a new column called next_month which is the sub_month + 1
# We then merge the dataframe with itself on the next_month and customerid
# We then filter out the rows where the next_month_y is null meaning that the customer has churned
# We then filter out the months that are not in the range of 2021-12 to 2022-03
# To map all data points, we add the months that are missing with 0 customer count
# We then plot the graph.



customer_df = main_df
customer_df = customer_df[customer_df['subscription'] != 'calendar purchase']
customer_df = customer_df[['customerid', 'sub_month']].drop_duplicates()
customer_df_base = customer_df
customer_df['next_month'] = customer_df['sub_month'] + 1
customer_df = customer_df.merge(customer_df_base,'left',left_on= ['next_month','customerid'],right_on= ['sub_month','customerid'])
customer_df = customer_df[customer_df['next_month_y'].isnull()]

customer_df = customer_df[(customer_df['next_month_x'] > '2021-11') & (customer_df['next_month_x'] < '2022-04')]
customer_df = customer_df.groupby('next_month_x')['customerid'].nunique().reset_index()
customer_df.rename(columns={'next_month_x':'churn_month', 'customerid':'customer_count'}, inplace=True)

included_months = ['2021-12', '2022-01', '2022-02', '2022-03']
included_months = [pd.Period(x, freq='M') for x in included_months]

for month in included_months:
    if month not in customer_df['churn_month'].values:
        customer_df = customer_df.append({'churn_month': month, 'customer_count': 0}, ignore_index=True)

customer_df = customer_df.sort_values(by=['churn_month']).reset_index(drop=True)



plt.plot(customer_df.index, customer_df['customer_count'])
plt.xlabel('Month')
plt.ylabel('Churned Customer Count')
plt.title('Monthly Churned Customer Count')
plt.xticks(range(0, len(customer_df.index)), customer_df['churn_month'], rotation=90)
plt.yticks(range(0,3,1))
plt.grid()
plt.show()

